In [2]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
import datetime as dt
from openbb_terminal.sdk import openbb

import webbrowser
import os

import pandas as pd
import quantstats as qs
import backtrader as bt

In [32]:
#download data to csv for backtrader
def openbb_data_to_bt_data(symbol, start_date, end_date):
    
    df = openbb.stocks.load(symbol, start_date=start_date, end_date=end_date)
    
    fn = f"{symbol.lower()}.csv"
    df.to_csv(fn)
    
    return bt.feeds.YahooFinanceCSVData(
        dataname=fn,
        fromdate=dt.datetime.strptime(start_date, '%Y-%m-%d'),
        todate=dt.datetime.strptime(end_date, '%Y-%m-%d')
    )

#This function downloads the data from the OpenBB SDK, converts it to a CSV, 
# and reads it into Backtrader’s YahooFinanceCSVData.

Build a backtest using Backtrader

Fund managers report their holdings every month. They don’t want to tell investors they lost money on the latest meme stock. So they sell them before the reporting deadline and buy higher-quality assets, like bonds.

You might be able to take advantage of this effect by buying bonds toward the end of the month and selling them at the beginning.

Start with a simple helper function that gets the last day of the month.

In [33]:
def last_day_of_month(any_day):
    # The day 28 exists in every month. 4 days later, it's always next month
    next_month = any_day.replace(day=28) + dt.timedelta(days=4)
    
    # subtracting the number of the current day brings us back one month
    return (next_month - dt.timedelta(days=next_month.day)).day

Next, setup the Backtrader strategy. All Backtrader strategies are built as classes and inherit bt.Strategy.

In [34]:
#First, set up the strategy parameters. end_of_month is the first day 
#I want to be long SPY and start_of_month is the last day I want to be short.

#The strategy logic is in the next method.
#This code tests if there’s a position in the market. 
#If not, it checks if the current day is within the first week of the month 
#and creates a short position. Otherwise, if the current day is within the last week, 
#it creates a long position.

#The last step closes the open positions.The strategy closes the long position 
#if the current day is no longer within the last week of the month. 
#The strategy closes the short position if the current day is no longer within 
#the first week of the month.

In [35]:
class MonthlyFlows(bt.Strategy):
    
    params = (
        ("end_of_month", 23),
        ("start_of_month", 7),
    )
    
    def __init__(self):
        self.order = None
        self.dataclose = self.datas[0].close
        
    def notify_order(self, order):
        # No more orders
        self.order = None    
    
    def next(self):
        
        # Get today's date, day of month, and last day of current month
        dt_ = self.datas[0].datetime.date(0)
        dom = dt_.day
        ldm = last_day_of_month(dt_)
        
        # If an order is pending, exit
        if self.order:
            return
        
        # Check if we are in the market
        if not self.position:
            
            # We're in the first week of the month, sell
            if dom <= self.params.start_of_month:
                # Sell the entire portfolio
                self.order = self.order_target_percent(target=-1)
                
                print(f"Created SELL of {self.order.size} at {self.data_close[0]} on day {dom}")
            
            # We're in the last week of the month, buy
            if dom >= self.params.end_of_month:
                # Buy the entire portfolio
                self.order = self.order_target_percent(target=1)
                
                print(f"Created BUY of {self.order.size} {self.data_close[0]} on day {dom}")
        
        # We are not in the market
        else:
            
            # If we're long
            if self.position.size > 0:
                
                # And not within the last week of the month, close
                if not self.params.end_of_month <= dom <= ldm:
                    
                    print(f"Created CLOSE of {self.position.size} at {self.data_close[0]} on day {dom}")
                    self.order = self.order_target_percent(target=0.0)
            
            # If we're short
            if self.position.size < 0:
                
                # And not within the first week of the month, close
                if not 1 <= dom <= self.params.start_of_month:
                    
                    print(f"Created CLOSE of {self.position.size} at {self.data_close[0]} on day {dom}")
                    # self.order = self.close()
                    self.order = self.order_target_percent(target=0.0)

In [36]:
#run backtest
data = openbb_data_to_bt_data(
    "SPY", 
    start_date="2013-01-03",
    end_date="2023-11-04"
)
cerebro = bt.Cerebro(stdstats=False)
cerebro.adddata(data)
cerebro.broker.setcash(1000.0)
cerebro.addstrategy(MonthlyFlows)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(
    bt.analyzers.Returns, _name="returns"
)
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="time_return"
)
backtest_result = cerebro.run()

Loading Daily data for SPY with starting period 2013-01-03.

Created SELL of -8 at 119.71 on day 3
Created CLOSE of -8 at 119.56 on day 8
Created BUY of 8 122.7 on day 23
Created CLOSE of 8 at 124.23 on day 1
Created SELL of -8 at 122.84 on day 4
Created CLOSE of -8 at 124.69 on day 8
Created BUY of 8 122.39 on day 25
Created CLOSE of 8 at 124.95 on day 1
Created SELL of -8 at 125.61 on day 4
Created CLOSE of -8 at 127.68 on day 8
Created BUY of 7 127.84 on day 25
Created CLOSE of 7 at 128.75 on day 1
Created SELL of -7 at 129.39 on day 2
Created CLOSE of -7 at 128.88 on day 8
Created BUY of 7 130.18 on day 23
Created CLOSE of 7 at 130.59 on day 1
Created SELL of -7 at 131.81 on day 2
Created CLOSE of -7 at 134.77 on day 8
Created BUY of 7 136.51 on day 23
Created CLOSE of 7 at 135.6 on day 3
Created SELL of -7 at 134.95 on day 4
Created CLOSE of -7 at 135.97 on day 10
Created BUY of 7 130.27 on day 24
Created CLOSE of 7 at 133.84 on day 1
Created SELL of -7 at 133.71 on day 2
Created CLOSE of -7 at 135.99 on day 8
Created BUY of 7 140.29 on day

The first step is to create a backtesting engine (Backtrader calls it Cerebro). Then add the data, initial cash, and strategy logic. Backtrader has built-in observers that track variables and performance throughout the backtest.

After setting up the backtest, run it.

The last step is to convert the results into a pandas DataFrame.

In [37]:
# Get the strategy returns as a dictionary
returns_dict= backtest_result[0].analyzers.time_return.get_analysis()

# Convert the dictionary to a DataFrame


In [38]:
# Convert the dictionary to a DataFrame
returns_df = pd.DataFrame(list(returns_dict.items()), columns=["date", "returns"])

# Convert the 'date' column to datetime
returns_df['date'] = pd.to_datetime(returns_df['date'])

# Set the 'date' column as the index
returns_df = returns_df.set_index('date')

In [39]:
returns_series = returns_df['returns']

In [40]:
print(type(returns_series))

<class 'pandas.core.series.Series'>


In [41]:
print(returns_series.head())

date
2013-01-03    0.000000
2013-01-04   -0.002640
2013-01-07    0.002647
2013-01-08    0.002720
2013-01-09   -0.002074
Name: returns, dtype: float64


 Assess the results using key performance metrics

Trading takes time, money, and effort. To make sure you’re better off not being long SPY, compare the strategy results to a long-only strategy.

QuantStats makes it easy.

QuantStats is a library jam-packed with trading performance and risk metrics. One of the best parts of Backtrader is that it works well with QuantStats.

In [42]:
bench = openbb.stocks.load(
    "SPY",
    start_date="2013-01-02",
    end_date="2023-11-03"
)["Adj Close"].pct_change()

if not isinstance(bench.index, pd.DatetimeIndex):
    bench.index = pd.to_datetime(bench.index)



Loading Daily data for SPY with starting period 2013-01-02.

In [43]:

# Generate the report
qs.reports.html(returns_series, benchmark_rets=bench, output="stats.html")
webbrowser.open('file://' + os.path.realpath('stats.html'))


True

True